# Patch daily with jura or kibo

In [ ]:
import os
import datetime
import pytz
import numpy as np
from unittest.mock import patch
from astropy.table import Table, join
from astropy.io import fits
# from desispec.io import read_table
from desiutil.log import get_logger, DEBUG
from desispec.io.meta import faflavor2program
from specprodDB.util import cameraid
from specprodDB.patch import get_options, get_data, patch_frames, patch_exposures, patch_tiles, patch_missing_frames_mjd

In [ ]:
with patch('sys.argv', ['patch_specprod', '--source', 'kibo', '--destination', 'daily', '--overwrite', os.environ['SCRATCH']]):
    options = get_options()
log = get_logger(DEBUG)
src, dst = get_data(options)

## QA on SURVEY, PROGRAM

## Do some QA on patch specprod

In [ ]:
for c in src['frames'].colnames:
    if hasattr(src['frames'][c], 'mask'):
        print(c)

In [ ]:
src['frames']['MJD'].min()

In [ ]:
src_exposures_bad_rows = list()
bad_columns = list()
for c in src['exposures'].colnames:
    if hasattr(src['exposures'][c], 'mask'):
        print(c)
        bad_columns.append(c)
        src_exposures_bad_rows.append(np.where(src['exposures'][c].mask)[0])
    elif src['exposures'][c].dtype.kind == 'f' and not np.isfinite(src['exposures'][c]).all():
        print(c)
        bad_columns.append(c)
        src_exposures_bad_rows.append(np.where(~np.isfinite(src['exposures'][c]))[0])
    else:
        pass
src_exposures_bad_rows
# src_exposures_bad_rows = np.unique(np.hstack(src_exposures_bad_rows))
# src['exposures'][src_exposures_bad_rows]

### All exposures for the tiles that contain bad exposures

In [ ]:
src_exposures_bad_tiles = np.in1d(src['exposures']['TILEID'], src['exposures']['TILEID'][src_exposures_bad_rows])
src['exposures'][src_exposures_bad_tiles]

In [ ]:
src_tiles_bad_rows = np.in1d(src['tiles']['TILEID'], src['exposures']['TILEID'][src_exposures_bad_rows])
src['tiles'][src_tiles_bad_rows]

### Consistency of SURVEY, PROGRAM, etc.

In [ ]:
for row in src['tiles']:
    w = np.where(src['exposures']['TILEID'] == row['TILEID'])[0]
    for c in ('SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        assert (src['exposures'][c][w] == row[c]).all()

In [ ]:
for row in src['exposures']:
    w = np.where(src['frames']['EXPID'] == row['EXPID'])[0]
    for c in ('SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        if c in src['frames'].colnames:
            assert (src['frames'][c][w] == row[c]).all()

## Find patch exposures not in daily, daily exposures not in patch

In [ ]:
assert (np.unique(src['exposures']['EXPID']) == sorted(src['exposures']['EXPID'])).all()
assert (np.unique(dst['exposures']['EXPID']) == sorted(dst['exposures']['EXPID'])).all()
assert (np.unique(src['frames']['EXPID']) == sorted(src['exposures']['EXPID'])).all()
assert (np.unique(dst['frames']['EXPID']) == sorted(dst['exposures']['EXPID'])).all()

In [ ]:
first_src_exposure, last_src_exposure = src['exposures']['EXPID'].min(), src['exposures']['EXPID'].max()
first_src_exposure, last_src_exposure

In [ ]:
first_src_night = src['exposures']['NIGHT'][src['exposures']['EXPID'] == first_src_exposure].min()
last_src_night = src['exposures']['NIGHT'][src['exposures']['EXPID'] == last_src_exposure].max()
first_src_night, last_src_night

In [ ]:
src['tiles']['LASTNIGHT'].min(), src['tiles']['LASTNIGHT'].max()

In [ ]:
src_expid_set = frozenset(src['exposures']['EXPID'].tolist())
dst_expid_set = frozenset(dst['exposures']['EXPID'].tolist())

In [ ]:
src_not_in_dst = src_expid_set - dst_expid_set
src_not_in_dst

In [ ]:
dst_not_in_src = dst_expid_set - src_expid_set
# dst_not_in_src

## Perform initial patching and QA

In [ ]:
timestamp = datetime.datetime.now(tz=pytz.timezone('US/Pacific'))
ymd = timestamp.strftime('%Y%m%d')
patched = dict()
patched['tiles_file'] = os.path.join(options.output, f'tiles-{options.dst}-patched-with-{options.src}-{ymd}.csv')
patched['exposures_file'] = os.path.join(options.output, f'exposures-{options.dst}-patched-with-{options.src}-{ymd}.fits')
patched['frames'] = patch_frames(src['frames'], dst['frames'])
patched['exposures'] = patch_exposures(src['exposures'], dst['exposures'])
patched['frames'] = patch_missing_frames_mjd(patched['exposures'], patched['frames'])
patched['tiles'] = patch_tiles(src['tiles'], dst['tiles'], timestamp)

### Are the values of `FAFLAVOR` and similar columns consistent among all tables?

For "historical" reasons the frames table does not contain `PROGRAM`.

In [ ]:
for row in patched['tiles']:
    w = np.where(patched['exposures']['TILEID'] == row['TILEID'])[0]
    for c in ('SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        if (patched['exposures'][c][w] != row[c]).any() and not (patched['exposures'][c][w] == 'unknown').all():
            print(c, row['TILEID'], row[c], patched['exposures'][c][w].tolist())

In [ ]:
for row in patched['exposures']:
    w = np.where(patched['frames']['EXPID'] == row['EXPID'])[0]
    for c in ('SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        if c in patched['frames'].colnames:
            if (patched['frames'][c][w] != row[c]).any():
                print(c, row['EXPID'], row[c], patched['frames'][c][w].tolist())

## "Back" patch exposures and frames with values of SURVEY, PROGRAM, ... from tiles.

In [ ]:
back_patch = {'tiles': 'exposures', 'exposures': 'frames'}
for s, d in back_patch.items():
    for row in patched[s]:
        key = 'TILEID' if s == 'tiles' else 'EXPID'
        w = np.where(patched[d][key] == row[key])[0]
        for column in ('SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
            if column in patched[d].colnames:
                if (patched[d][column][w] != row[column]).any():
                    log.info("Patching %s associated with %s %d with %s = '%s'.", d, ('tile' if s == 'tiles' else 'exposure'), row[key], column, row[column])
                    patched[d][column][w] = row[column]

### Re-run QA

In [ ]:
for s, d in back_patch.items():
    for row in patched[s]:
        key = 'TILEID' if s == 'tiles' else 'EXPID'
        w = np.where(patched[d][key] == row[key])[0]
        for column in ('SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
            if column in patched[d].colnames:
                assert (patched[d][column][w] == row[column]).all()